<a href="https://colab.research.google.com/github/SireeshaM6/Advanced_House_Dataset/blob/main/Bidirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

from tensorflow.keras.layers import Embedding,Dense,LSTM,Bidirectional,Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,classification_report

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = pd.read_csv("/content/fake_news_data.csv",error_bad_lines=False,engine="python")
data.head()

<ipython-input-2-9e83da879feb>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/fake_news_data.csv",error_bad_lines=False,engine="python")


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
# Divide the dataset
X = data.drop(['label'],axis=1)
X.head()

In [ ]:
y = data['label']

In [ ]:
X.shape,y.shape

In [ ]:
messages = X.copy()

In [ ]:
# Resetting the index of the messages

messages.reset_index(inplace=True)

**Text Preprocessing**

In [ ]:
ps = PorterStemmer()
corpus = []

for i in range(len(messages)):
  review = re.sub('[^a-zA-z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus[0]

**One Hot Encoding Representations**

In [ ]:
vocab_size = 5000 # Unique words in a dictionary

In [ ]:
onehot_re = [one_hot(word,vocab_size) for word in corpus]
onehot_re[0]

In [ ]:
# Padding to the words
sent_length = 20
pad_docs = pad_sequences(onehot_re,padding='post',maxlen=sent_length)

In [ ]:
pad_docs[0]

In [ ]:
# Creating the model
fea_dim= 40
model = Sequential()
model.add(Embedding(vocab_size,40,input_length=sent_length))#
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
# Converting into Arrays
X_final = np.array(pad_docs)
y_final = np.array(y)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)
print('Completed...')

In [ ]:
X_train.shape,X_test.shape

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y